In [4]:
# Download dataset from Kaggle using KaggleHub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pauloarayasantiago/pokmon-stats-across-generations-and-typings")

print("Path to dataset files:", path)

100%|██████████| 103k/103k [00:00<00:00, 485kB/s]

Extracting files...
Path to dataset files: /Users/tongluyangyu/.cache/kagglehub/datasets/pauloarayasantiago/pokmon-stats-across-generations-and-typings/versions/1


In [6]:
# Read dataset
import pandas as pd
dataset = pd.read_csv(path + "/pokemon_dataset.csv")
print(dataset.head())

   pokemon_id        name primary_type secondary_type first_appreance  \
0           1   bulbasaur        grass         poison        red/blue   
1           2     ivysaur        grass         poison        red/blue   
2           3    venusaur        grass         poison        red/blue   
3           4  charmander         fire            NaN        red/blue   
4           5  charmeleon         fire            NaN        red/blue   

  generation category  total_base_stats  hp  attack  defense  special_attack  \
0      gen 1  regular               318  45      49       49              65   
1      gen 1  regular               405  60      62       63              80   
2      gen 1  regular               525  80      82       83             100   
3      gen 1  regular               309  39      52       43              60   
4      gen 1  regular               405  58      64       58              80   

   special_defense  speed  
0               65     45  
1               80     6